In [15]:
from shapely.geometry import Polygon
import geopandas as gp

In [16]:
path1 = '/Users/saifbhatti/Downloads/cb_2018_us_county_500k/cb_2018_us_county_500k.shp' #US county shapefile
path2 = '/Users/saifbhatti/Downloads/mygeodata/map1-polygon.shp' #e85 stations
poly1 = gp.read_file(path1)
poly2 = gp.read_file(path2)

In [ ]:
inter = gp.overlay(poly1, poly2, how='intersection')
inter